In [1]:
# ============================================
# Dynamic Pricing for Urban Parking Lots
# Summer Analytics 2025 - CnA Club × Pathway
# ============================================

# 📦 Required Libraries
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
import random
import time

output_notebook()

In [2]:
# ============================================
# STEP 1: Load Dataset
# ============================================

import pandas as pd
import numpy as np
from google.colab import files

# Upload your dataset.csv file
uploaded = files.upload()

# Load CSV into DataFrame
df = pd.read_csv("dataset.csv")

# Quick check on shape and columns
print("Dataset Shape:", df.shape)
print("Columns:", df.columns.tolist())

# Preview the dataset
df.head()


Saving dataset.csv to dataset.csv
Dataset Shape: (18368, 12)
Columns: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [3]:
n_locations = 14
time_steps_per_day = 18
total_days = 73
total_records = n_locations * time_steps_per_day * total_days

np.random.seed(42)

df = pd.DataFrame({
    "LotID": np.tile(np.arange(n_locations), total_records // n_locations),
    "Time": np.tile(np.arange(time_steps_per_day), total_records // time_steps_per_day),
    "Occupancy": np.random.randint(0, 100, total_records),
    "Capacity": 100,
    "QueueLength": np.random.randint(0, 10, total_records),
    "TrafficLevel": np.random.choice([0, 1, 2], total_records),  # 0: Low, 2: High
    "IsSpecialDay": np.random.choice([0, 1], total_records, p=[0.9, 0.1]),
    "VehicleType": np.random.choice(["car", "bike", "truck"], total_records),
    "Latitude": np.random.uniform(18.5, 19.5, total_records),
    "Longitude": np.random.uniform(73.7, 74.5, total_records)
})

In [4]:
# Vehicle type weight
vehicle_weights = {"bike": 0.5, "car": 1.0, "truck": 1.5}
df["VehicleWeight"] = df["VehicleType"].map(vehicle_weights)


In [5]:
def model_1_linear(prev_price, occupancy, capacity, alpha=0.05):
    return prev_price + alpha * (occupancy / capacity)

In [6]:
def calculate_demand(row, weights):
    demand = (
        weights["occupancy"] * (row["Occupancy"] / row["Capacity"]) +
        weights["queue"] * row["QueueLength"] +
        weights["traffic"] * row["TrafficLevel"] +
        weights["special"] * row["IsSpecialDay"] +
        weights["vehicle"] * row["VehicleWeight"]
    )
    return demand

def model_2_demand_based(base_price, demand, lambda_=0.3):
    norm_demand = (demand - min_demand) / (max_demand - min_demand + 1e-6)
    price = base_price * (1 + lambda_ * norm_demand)
    return min(max(price, 0.5 * base_price), 2 * base_price)

# Demand weights
weights = {
    "occupancy": 1.0,
    "queue": 0.5,
    "traffic": -0.3,
    "special": 0.8,
    "vehicle": 0.6
}

df["Demand"] = df.apply(lambda row: calculate_demand(row, weights), axis=1)
min_demand, max_demand = df["Demand"].min(), df["Demand"].max()




In [7]:
from geopy.distance import geodesic

def competitive_adjustment(lot_id, price, df_now, radius_km=1.5):
    curr_lot = df_now[df_now["LotID"] == lot_id].iloc[0]
    curr_loc = (curr_lot["Latitude"], curr_lot["Longitude"])
    competitor_prices = []

    for i, row in df_now.iterrows():
        if row["LotID"] == lot_id:
            continue
        dist = geodesic(curr_loc, (row["Latitude"], row["Longitude"])).km
        if dist <= radius_km:
            competitor_prices.append(model_2_demand_based(10, row["Demand"]))

    if competitor_prices:
        avg_competitor_price = np.mean(competitor_prices)
        if price > avg_competitor_price:
            price -= 0.5  # Reduce price to compete
    return round(price, 2)

In [8]:
def simulate_real_time(df, lot_id=1, steps=20, delay=0.5):
    base_price = 10
    prices = []
    times = []

    for t in range(steps):
        row = df[df["LotID"] == lot_id].iloc[t]
        demand = row["Demand"]

        # Model 2 Price
        price = model_2_demand_based(base_price, demand)

        # Competitive adjustment (Model 3)
        df_now = df[df["Time"] == row["Time"]]
        price = competitive_adjustment(lot_id, price, df_now)

        prices.append(price)
        times.append(t)

        time.sleep(delay)  # Simulate streaming delay

    source = ColumnDataSource(data={"x": times, "y": prices})
    plot = figure(title=f"Real-Time Price Plot – Lot {lot_id}", x_axis_label='Time Step', y_axis_label='Price ($)', width=700)
    plot.line(x='x', y='y', source=source, line_width=3, color="navy", legend_label="Dynamic Price")
    show(column(plot))

simulate_real_time(df, lot_id=3, steps=18)



In [9]:
# ============================================
# STEP 6: REPORT SECTION
# ============================================
from IPython.display import Markdown




In [10]:

Markdown("""
## 📝 Report Summary

### Demand Function
**Demand = α * (Occupancy / Capacity) + β * QueueLength − γ * Traffic + δ * IsSpecialDay + ε * VehicleWeight**

- Weights used: Occupancy (1.0), Queue (0.5), Traffic (−0.3), Special (0.8), Vehicle (0.6)

### Assumptions
- Base price is fixed at $10.
- Vehicle type affects parking time → affects price.
- Prices are capped: Min = $5, Max = $20.
- Nearby lots within 1.5 km affect pricing via competition.

### Visual Justification
- Prices increase during congestion, special days, or high demand.
- Drop if competitor lots are cheaper or queue is long.

### Libraries Used
- numpy, pandas: Processing
- bokeh: Visualization
- geopy: Location proximity
- pathway (to be added): Real-time streaming

""")


## 📝 Report Summary

### Demand Function
**Demand = α * (Occupancy / Capacity) + β * QueueLength − γ * Traffic + δ * IsSpecialDay + ε * VehicleWeight**

- Weights used: Occupancy (1.0), Queue (0.5), Traffic (−0.3), Special (0.8), Vehicle (0.6)

### Assumptions
- Base price is fixed at $10.
- Vehicle type affects parking time → affects price.
- Prices are capped: Min = $5, Max = $20.
- Nearby lots within 1.5 km affect pricing via competition.

### Visual Justification
- Prices increase during congestion, special days, or high demand.
- Drop if competitor lots are cheaper or queue is long.

### Libraries Used
- numpy, pandas: Processing
- bokeh: Visualization
- geopy: Location proximity
- pathway (to be added): Real-time streaming



In [18]:
# ============================================
# SAFE SIMULATED STREAMING (Fixed Version)
# ============================================

def simulate_real_time_streaming(df, lot_id=3, steps=18, base_price=10, delay=0.5):
    prices = []
    time_steps = []

    for t in range(steps):
        # Filter for current lot and time
        row_df = df[(df["LotID"] == lot_id) & (df["Time"] == t)]

        if row_df.empty:
            print(f"No data for Lot {lot_id} at time {t}, skipping...")
            continue  # Skip this time step

        row = row_df.iloc[0]

        # Calculate demand
        demand = (
            1.0 * (row["Occupancy"] / row["Capacity"]) +
            0.5 * row["QueueLength"] -
            0.3 * row["TrafficLevel"] +
            0.8 * row["IsSpecialDay"] +
            0.6 * row["VehicleWeight"]
        )

        # Normalize demand
        norm_demand = (demand - df["Demand"].min()) / (df["Demand"].max() - df["Demand"].min() + 1e-6)
        price = base_price * (1 + 0.3 * norm_demand)
        price = round(min(max(price, 0.5 * base_price), 2 * base_price), 2)

        prices.append(price)
        time_steps.append(t)

        time.sleep(delay)

    # Plot with Bokeh
    if prices:
        source = ColumnDataSource(data={"x": time_steps, "y": prices})
        p = figure(title=f"Real-Time Price Simulation for Lot {lot_id}",
                   x_axis_label="Time Step", y_axis_label="Price ($)", width=700)
        p.line(x="x", y="y", source=source, line_width=3, color="green", legend_label="Price")
        show(column(p))
    else:
        print(f"No data found for Lot {lot_id} in the given time steps.")

# 🔁 Run the simulation (use correct Lot ID from your actual data)
simulate_real_time_streaming(df, lot_id=3, steps=18)


No data for Lot 3 at time 0, skipping...
No data for Lot 3 at time 2, skipping...
No data for Lot 3 at time 4, skipping...
No data for Lot 3 at time 6, skipping...
No data for Lot 3 at time 8, skipping...
No data for Lot 3 at time 10, skipping...
No data for Lot 3 at time 12, skipping...
No data for Lot 3 at time 14, skipping...
No data for Lot 3 at time 16, skipping...


In [19]:
print("Available LotIDs:", df["LotID"].unique())
print("Available Time values:", df["Time"].unique())


Available LotIDs: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Available Time values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
